# Evaluation of a Model

In [1]:
import os

import e2cnn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import PIL

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser as ConfigParser

import utils
# Ipmport various network architectures
from networks import AGRadGalNet, DNSteerableLeNet, DNSteerableAGRadGalNet #e2cnn module only works in python3.7+
# Import various data classes
from datasets import FRDEEPF
from datasets import MiraBest_full, MBFRConfident, MBFRUncertain, MBHybrid
from datasets import MingoLoTSS, MLFR, MLFRTest

from sklearn.metrics import classification_report, roc_curve, auc

# Set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)

# Get correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read in config file
config_name = ["bowles2021mirabest.cfg", 
               "bowles2021mingo.cfg", 
               "scaife2021mirabestDN.cfg", 
               "scaife2021mingo.cfg", 
               "e2attentionmirabest.cfg"
              ]
config_name = "configs/"+config_name[-1]
config = ConfigParser.ConfigParser(allow_no_value=True)
config.read(config_name)

# Load network architecture (with random weights)
print(f"Loading in {config['model']['base']}")
net = locals()[config['model']['base']](**config['model']).to(device)

Loading in DNSteerableAGRadGalNet


/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:61: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  sampled_basis = sampled_basis[mask, ...]


In [2]:
added_path = "/search_lr_1e-05/"
csv_path = config['output']['directory'] + added_path + config['output']['training_evaluation']
df = pd.read_csv(csv_path)
df.head(20)

,train_loss,validation_loss,TP,FP,FN,TN,validation_update
0,0.692433,0.849427,26.0,33.0,4.0,5.0,True
1,0.706157,0.839778,29.0,35.0,1.0,3.0,True
2,0.710593,0.842674,30.0,36.0,0.0,2.0,False
3,0.691907,0.842423,27.0,33.0,3.0,5.0,False
4,0.702030,0.886265,29.0,38.0,1.0,0.0,False
5,0.717401,0.842676,28.0,34.0,2.0,4.0,False
6,0.704911,0.842630,28.0,34.0,2.0,4.0,False
7,0.715000,0.828648,25.0,30.0,5.0,8.0,True
8,0.672473,0.774524,25.0,26.0,5.0,12.0,True
9,0.723125,0.871686,28.0,36.0,2.0,2.0,False


In [3]:
best = df.iloc[list(df['validation_update'])].iloc[-1]
best_epoch = int(best.name)
print(best_epoch)
print(best)

351
train_loss           0.685894
validation_loss      0.607424
TP                       19.0
FP                        9.0
FN                       11.0
TN                       29.0
validation_update        True
Name: 351, dtype: object


In [4]:
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

configs = [
    "e2attentionmirabest.cfg",# - Fully tested
    "e2attentionmingo.cfg",
    "bowles2021mirabest.cfg",
    "bowles2021mingo.cfg",
    "scaife2021mirabest.cfg",
    "scaife2021mingo.cfg"
]
data_configs = [
    "e2attentionmirabest.cfg", # Mirabest Dataset - MBFR
    "e2attentionmingo.cfg" # Mingo Dataset - MLFR
]
augmentations = [
    "rotation and flipping",
    "random rotation",
    "restricted random rotation"
]

for cfg in configs:
    config = ConfigParser.ConfigParser(allow_no_value=True)
    data_config = ConfigParser.ConfigParser(allow_no_value=True)
    config.read('configs/'+cfg)
    csv_path = config['output']['directory'] +'/'+ config['output']['training_evaluation']
    df = pd.read_csv(csv_path)
    best = df.iloc[list(df['validation_update'])].iloc[-1]

    net = locals()[config['model']['base']](**config['model']).to(device)
    model = utils.utils.load_model(config, load_model='best', device=device)
    

    for d_cfg in data_configs:
        for augmentation in augmentations:
            data_config.read('configs/'+d_cfg)
            print(f"Evaluating: {config['model']['base']}\t{data_config['data']['dataset']}\t{augmentation}")
            data  = utils.data.load(
                data_config, 
                train=False, 
                augmentation=augmentation, 
                data_loader=True
            )
            
            y_pred, y_labels = utils.evaluation.predict(
                model, 
                data, 
                augmentation_loops=100, 
                raw_predictions=True
            )
            
            utils.evaluation.save_evaluation(
                y_pred, 
                y_labels,
                model_name=config['model']['base'],
                train_data=config['data']['dataset'],
                train_augmentation="rotation and flipping",
                test_data=data_config['data']['dataset'],
                test_augmentation=augmentation,
                epoch=int(best.name),
                best=True,
                raw=False,
                PATH='./'
            )

Evaluating: DNSteerableAGRadGalNet	MBFRUncertain	rotation and flipping
Files already downloaded and verified


/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1314: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(
/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Evaluating: DNSteerableAGRadGalNet	MBFRUncertain	random rotation
Files already downloaded and verified


/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1314: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(
/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torch/nn/functional.py:3451: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Evaluating: DNSteerableAGRadGalNet	MBFRUncertain	restricted random rotation
Files already downloaded and verified


...

Files already downloaded and verified
Evaluating: DNSteerableLeNet	MLFR	random rotation


/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1314: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


Files already downloaded and verified
Evaluating: DNSteerableLeNet	MLFR	restricted random rotation


/raid/scratch/mbowles/EquivariantSelfAttention/venv/lib/python3.8/site-packages/torchvision/transforms/transforms.py:1314: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


Files already downloaded and verified
